

---



---



**Référence :** https://github.com/facebookresearch/XLM

# **Applications: Supervised / Unsupervised MT**

In [1]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME
%env OUTPATH /home/XLM/data/30000/es-fr

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM
env: OUTPATH=/home/XLM/data/30000/es-fr


In [2]:
%env exp_id=hypothesis
#%env batch_size=32
%env batch_size=16
%env lgs=es-fr
# Maximum length of sentences (after BPE)
%env max_len=100

env: exp_id=hypothesis
env: batch_size=16
env: lgs=es-fr
env: max_len=100


In [12]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

main_path = "/home/XLM/data/30000/"

import io

def n_lines(file_path):
    i = 0
    with open(file_path, mode = "r", encoding='UTF-8') as myfile :
        for _ in myfile.readlines() :
            i = i + 1
    return i

tensor_len = n_lines(file_path = main_path + "es-fr/es-fr.es.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 38067593


In [13]:
tensor_len = 38067593
batch_size = 16

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

    
epoch_size = getEpochSize(tensor_len = tensor_len, batch_size = batch_size)
print("epoch_size = " + str(epoch_size))

epoch_size = 2379225


In [3]:
# Et deduire epoch_size (evaluation frequency, -1 for parallel data size)
%env epoch_size=2379225
#%env epoch_size=-1

env: epoch_size=2379225


In [4]:
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [5]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1


**Pretrain a language model (MLM+TLM)**

In [16]:
def get_max_epoch(tensor_len):
    return int(100*(tensor_len/139670)) # par expérience

def get_nb_hour(max_epoch, nb_gpu = 1):
    return int(5*(max_epoch/100)) * nb_gpu

max_epoch = get_max_epoch(tensor_len)
print("max_epoch = "+ str(max_epoch))
print("nb_hour = " + str(get_nb_hour(max_epoch, nb_gpu = 1)))

max_epoch = 27255
nb_hour = 1362


In [6]:
%env max_epoch=27255

env: max_epoch=27255


In [7]:
# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false
%env lgs=es-fr
%env mlm_steps=es,fr,es-fr

env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false
env: lgs=es-fr
env: mlm_steps=es,fr,es-fr


In [ ]:
%env max_epoch=1

%env use_memory=True
%env mem_enc_positions=0
%env mem_dec_positions=0
#--use_memory $use_memory --mem_enc_positions $mem_enc_positions --mem_dec_positions $mem_dec_positions

! python train.py --exp_name esfr_mlm_tlm --dump_path ./dumped/ --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --exp_id $exp_id --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --use_memory $use_memory --mem_enc_positions $mem_enc_positions --mem_dec_positions $mem_dec_positions

env: max_epoch=1
env: use_memory=True
env: mem_enc_positions=0
env: mem_dec_positions=0
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm
INFO - 05/10/20 21:04:16 - 0:00:00 - ============ Initialized logger ============
INFO - 05/10/20 21:04:16 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 16
                                     beam_size: 1
                                     bptt: 256
                          

In [ ]:
# Si l'entrainement echoue
#! rm -R dumped/esfr_mlm_tlm



---



---



**Train on unsupervised MT from a pretrained model**

In [ ]:
#%env batch_size=...
#%env epoch_size=...

%env eval_bleu=true
# comme eval_bleu=true
%cd /home/jupyter/meta_XLM/XLM
! chmod +x src/evaluation/multi-bleu.perl

In [ ]:
#%env stopping_criterion=valid_es-fr_mt_bleu,10
%env stopping_criterion=valid_es-fr_mt_bleu,2
%env validation_metrics=valid_es-fr_mt_bleu
# reload encoder and decoder from dumped/esfr_mlm_tlm/hypothesis/best-valid_mlm_ppl.pth
%env reload_model=dumped/esfr_mlm_tlm/hypothesis/best-valid_mlm_ppl.pth,dumped/esfr_mlm_tlm/hypothesis/best-valid_mlm_ppl.pth
%env ae_steps=es,fr
%env bt_steps=es-fr-es,fr-es-fr

In [ ]:
#! python train.py --exp_name unsupMT_esfr --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --exp_id $exp_id  
%env mt_steps=es-fr,fr-es
! python train.py --exp_name unsupMT_esfr --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --exp_id $exp_id  

In [ ]:
# Si l'entrainement echoue
#! rm -R dumped/unsupMT_esfr